In [1]:
#Bora Aslan
#2220356080
import numpy as np # for arrays
import pandas as pd # to read csv as dataframe
import math
import random

In [2]:
data = pd.read_csv("16P.csv", encoding="iso8859")

data = data.sample(frac=1).reset_index(drop=True)

data.drop(["Response Id"], axis=1, inplace=True)

pNames = {"ESTJ": 0, "ENTJ": 1, "ESFJ": 2, "ENFJ": 3, "ISTJ": 4, "ISFJ": 5, "INTJ": 6, "INFJ": 7, "ESTP": 8, "ESFP": 9,
          "ENTP": 10, "ENFP": 11, "ISTP": 12, "ISFP": 13, "INTP": 14, "INFP": 15}

data['numericMBTI'] = data['Personality'].map(pNames)

data.drop(columns=['Personality'], inplace=True)

#normalizing data which is -3 to 3 by simple math
#except numericMBTI column obviously!
data.iloc[:,:-1] = ((data.iloc[:,:-1] +3) / (6))




In [3]:
def cal_distance(a, b):
    distance = np.sqrt(np.sum((a - b) ** 2, axis=1))

    return distance

In [4]:
X_train = data.iloc[:int(len(data) * 0.1), :-1]
y_train = data.iloc[:int(len(data) * 0.1), -1:]

last_row = X_train.iloc[-1:, :]
last_rowy = y_train.iloc[-1:, :]

X_train = pd.concat([X_train, last_row], axis=0, ignore_index=True)
y_train = pd.concat([y_train, last_rowy], axis=0, ignore_index=True)

X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

X_train_list = np.array_split(X_train_np, 5)
y_train_list = np.array_split(y_train_np, 5)


In [7]:
for K in [1, 3, 5, 7, 9]:# to execute the code for each K value
    
    matrix = [[0 for i in range(16)] for i in range(16)]
    #confusion matrix created to store results
    
    for i in range(5):
        #for cross validation 4 piece of train lists gets together again as train piece and 
        #remaining piece is test piece
        fourpiece = X_train_list[:i] + X_train_list[i + 1:]
        test_np = np.concatenate(fourpiece, axis=0)

        f0urpiece = y_train_list[:i] + y_train_list[i + 1:]
        y_train_np = np.concatenate(f0urpiece, axis=0)
        
        
        for j in range(len(X_train_list[i])):
            testline = X_train_list[i][j]
            testliney = y_train_list[i][j]
            distance = cal_distance(testline, test_np)
            k_neighbors = y_train_np[np.argsort(distance)[:K]]
            arr_1d = np.ravel(k_neighbors)
            
            counts = np.bincount(arr_1d)
            prediction = np.argwhere(counts == counts.max()).flatten()

            if len(prediction) != 1:
                #print(prediction) DELETE THE # TO SEE KNN TIES
                prediction = prediction[0]
            matrix[testliney.item()][prediction.item()] += 1
            
    #calculating TP,TN,FP,FN to derive precision,recall and accuracy
    total = 0
    for i in range(16):#runs for every class once
        
        for l in range(16):#calculates the amount of entries to matrix to find the size
            
            total += matrix[i][l]
    #general variables set to 0
    recallT = 0
    precisionT = 0
    accuracynominatorT = 0
    
    for i in range(16):# this i is used as row index for FN, column index for FP
        TP = matrix[i][i]
        FN = 0
        FP = 0
        for j in range(16):# this j is used as column index for FN, row index for FP
            if i == j:
                continue
            FN += matrix[i][j]
            FP = matrix[j][i]

        TN = total - FN - FP - TP

        precision = TP / (TP + FP)

        precisionT += precision

        recall = TP / (TP + FN)

        recallT += recall

        accuracynominatorT += TP + TN

    print("K Parameter is {}, results:".format(K))
    print("Recall is {:.4f}%".format(100 * recallT / 16))
    print("Precision is {:.4f}%".format(100 * precisionT / 16))
    print("Accuracy is {:.4f}%".format(100 * accuracynominatorT / (total * 16)))
    print("---------------------------")
#Error Analysis for Classification
#During the implementation of my KNN classifier, I encountered an issue where there were instances where the closest k points
#belonged to multiple classes, resulting in ties. This is due to the nature of the KNN algorithm, which relies on the distance
#between points to determine the class of a given instance and could be revialed if the input questions are just
#aligned in a way. You can see the ties that happens by going up lines and removing the # i put on line tht catchs ties and 
#picks the closest class as predicted.

K Parameter is 1, results:
Recall is 96.7094%
Precision is 99.7066%
Accuracy is 99.7771%
---------------------------
K Parameter is 3, results:
Recall is 98.1735%
Precision is 99.8981%
Accuracy is 99.8802%
---------------------------
K Parameter is 5, results:
Recall is 98.4044%
Precision is 99.8786%
Accuracy is 99.8937%
---------------------------
K Parameter is 7, results:
Recall is 98.5740%
Precision is 99.9314%
Accuracy is 99.9073%
---------------------------
K Parameter is 9, results:
Recall is 98.6281%
Precision is 99.9315%
Accuracy is 99.9104%
---------------------------
